In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

from core.utils.device import DEVICE

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
model.to(DEVICE)
model.eval()
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

In [3]:
import numpy as np

from core.gather_data.hidden_space import collect_hidden_space_by_language
from core.visualize.process_hidden_space import extract_pca_components, project_onto_pca
from core.visualize.visualize_projections import visualize_explained_variance, visualize_projections
from core.preprocess_data.flores_plus import load_flores_plus

train_df, test_df = load_flores_plus(["eng_Latn", "rus_Cyrl"], {"eng_Latn": "en", "rus_Cyrl": "ru"}, train_size=50)

test_df = test_df[:100]

hidden_space_by_language_train, token_map_train = collect_hidden_space_by_language(model, tokenizer, train_df)
combined_embeddings = np.concat(list(hidden_space_by_language_train.values()), axis=1)
pca_components, pca_means, explained_variance_ratios = extract_pca_components(combined_embeddings)
visualize_explained_variance(explained_variance_ratios)

projections_train = project_onto_pca(hidden_space_by_language_train, pca_components, pca_means)
visualize_projections(hidden_space_by_language_train, token_map_train, projections_train, tokenizer)

hidden_space_by_language_test, token_map_test = collect_hidden_space_by_language(model, tokenizer, test_df)
projections_test = project_onto_pca(hidden_space_by_language_test, pca_components, pca_means)
visualize_projections(hidden_space_by_language_test, token_map_test, projections_test, tokenizer)

ModuleNotFoundError: No module named 'core.visualize.process_hidden_space'